In [1]:
import sys
import os

# Get the project root directory (parent of the notebooks folder)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the src folder to the Python path
sys.path.append(project_root)

In [2]:
from src.data_preprocessing import DataLoaders

loaders = DataLoaders(trial_data=True)
train_dataloader = loaders.get_train_dataloader()
test_dataloader = loaders.get_test_dataloader()
# for images, labels in train_dataloader:
#     print(f'Batch of images shape: {images.shape}')
#     print(f'Batch of labels: {labels}')

# for images, labels in test_dataloader:
#     print(f'Batch of images shape: {images.shape}')
#     print(f'Batch of labels: {labels}')

In [3]:

import torch.optim as optim
import torch.nn as nn
import torch
import json
import wandb
from torchmetrics import Accuracy, F1Score
from src.models.tiny_vit import tiny_vit_5m_224

num_epochs = 10
learning_rate = 2.5e-4
with open('../data/country_to_index_mapped.json', 'r') as f:
    num_classes = len(json.load(f))

model = tiny_vit_5m_224(pretrained=True)


criterion = nn.CrossEntropyLoss()


device = torch.device("mps")
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-8)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-5)
f1_metric = F1Score(task='multiclass', num_classes=num_classes).to(device)
accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")
# Assuming parameters are of type float32 (4 bytes)
param_size_in_bytes = total_params * 4
# Convert to megabytes
param_size_in_mb = param_size_in_bytes / (1024 ** 2)
print(f"Model size: {param_size_in_mb:.2f} MB")

wandb.init(
    project="geoguessr AI",
    config={
    "learning_rate": learning_rate,
    "architecture": "TinyViT-5M-224",
    "dataset": "street-location-images- data-mapped",
    "epochs": num_epochs,
    "model_parameters": total_params,
    "model_size": param_size_in_mb
    }
)


/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:640: UserWarning: Overwriting tiny_vit_5m_224 in registry with src.models.tiny_vit.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:653: UserWarning: Overwriting tiny_vit_11m_224 in registry with src.models.tiny_vit.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/Users/mateuszwitka-jezewski/Documents/Projekty/geoguessrai/src/models/tiny_vit.py:666: UserWarning: Overwriting tiny_vit_21m_224 in registry with src.models.tiny_vit.tiny_vit_21m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  @register_model
/Users/mateuszwitka-jezewski/Documents/Projekty/geogue

NameError: name 'torch' is not defined

In [8]:
import torch
from torchmetrics import Accuracy, F1Score

def evaluate(model, test_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0

    accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes).to(device)
    f1_metric = F1Score(task='multiclass', num_classes=num_classes).to(device)  # For multiclass classification

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)

            accuracy_metric.update(predicted, labels)
            f1_metric.update(predicted, labels)

    avg_loss = test_loss / len(test_loader)
    accuracy = accuracy_metric.compute().item() * 100  # Convert to percentage
    f1_score = f1_metric.compute().item()  # Final F1 score

    return avg_loss, accuracy, f1_score


In [10]:
from tqdm import tqdm

model.to(device)
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for batch_idx, (inputs, labels) in enumerate(tqdm(train_dataloader)):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)

        # Update metrics (both accuracy and F1 score)
        accuracy_metric.update(predicted, labels)
        f1_metric.update(predicted, labels)

    # Compute average loss for training
    train_loss = running_loss / len(train_dataloader)
    train_accuracy = accuracy_metric.compute().item() * 100  # Convert to percentage
    train_f1_score = f1_metric.compute().item()  # F1 score

    # Evaluate on the test set
    test_loss, test_accuracy, test_f1_score = evaluate(model, test_dataloader, criterion, device)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, '
          f'Train Accuracy: {train_accuracy:.2f}%, '
          f'Train F1 Score: {train_f1_score:.4f}, '
          f'Test Loss: {test_loss:.4f}, '
          f'Test Accuracy: {test_accuracy:.2f}%, '
          f'Test F1 Score: {test_f1_score:.4f}')

    wandb.log({"train_loss": train_loss, "test_loss": test_loss, "train_accuracy": train_accuracy, "test_accuracy": test_accuracy, "train_f1_score": train_f1_score, "test_f1_score": test_f1_score, "epoch": epoch})
wandb.finish()


100%|██████████| 1/1 [00:25<00:00, 25.64s/it]


Epoch [1/10], Train Loss: 8.3600, Train Accuracy: 0.00%, Test Loss: 8.3828, Test Accuracy: 0.00%


100%|██████████| 1/1 [00:09<00:00,  9.43s/it]


Epoch [2/10], Train Loss: 5.8863, Train Accuracy: 0.00%, Test Loss: 8.1654, Test Accuracy: 0.00%


100%|██████████| 1/1 [00:09<00:00,  9.93s/it]


...

100%|██████████| 1/1 [00:07<00:00,  7.58s/it]


Epoch [9/10], Train Loss: 0.0554, Train Accuracy: 100.00%, Test Loss: 6.9577, Test Accuracy: 0.00%


100%|██████████| 1/1 [00:07<00:00,  7.98s/it]


Epoch [10/10], Train Loss: 0.0288, Train Accuracy: 100.00%, Test Loss: 6.9327, Test Accuracy: 0.00%


test_accuracy,▁▁▁▁▁▁▁▁▁▁
test_loss,█▇▆▅▄▃▂▁▁▁
train_accuracy,▁▁▃▅▇█████
train_loss,█▆▄▃▂▂▁▁▁▁
test_accuracy,0.0
test_loss,6.93268
train_accuracy,100.0
train_loss,0.02881


In [ ]:
import torch
import torch
from transformers import ViTConfig
from huggingface_hub import login, Repository
from dotenv import load_dotenv
import os
import shutil

load_dotenv()


model_save_path = "../data/models/"
repo = Repository(local_dir=model_save_path, clone_from="mwitjez/geoguessr_tiny_ViT")

torch.save(model, f"{model_save_path}/geoguessr_model.bin")

config = ViTConfig()
config.save_pretrained(model_save_path)

repo.push_to_hub()
if os.path.exists(model_save_path):
    shutil.rmtree(model_save_path)
    print(f"'{model_save_path}' has been removed successfully.")
else:
    print(f"'{model_save_path}' does not exist.")
